In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


#  %load_ext rpy2.ipython

In [123]:
df = pd.read_csv('UCI_CC_defaulting.csv')
df.head()
df = df.drop(df.columns[0], axis=1)
new_columns = df.iloc[0]
df = df.drop(0)
# Rename columns
df.columns = new_columns

df = df.reset_index(drop=True)
display(df)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,3,1,39,0,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,150000,1,3,2,43,-1,-1,-1,-1,0,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,30000,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,80000,1,3,1,41,1,-1,0,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [124]:
# Obtain our X and Y values
y = df['default payment next month']
y = [int(Y) for Y in y]
X = df.drop('default payment next month', axis=1)
d1 = X.copy()
d2 = X.copy()
d3 = X.copy()
d4 = X.copy()
# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# We have one matrix of values for each of our data sets

In [125]:
# Include all features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


Not evaluating on any protected features 

In [126]:
cols = ['EDUCATION', 'MARRIAGE', 'SEX', 'AGE']
d1 = d1.drop(cols, axis=1)
X_train1, X_test1, y_train1, y_test1 = train_test_split(d1, y, test_size=0.3)


Not considering marriage

In [127]:
d2 = d2.drop('MARRIAGE', axis=1)
X_train2, X_test2, y_train2, y_test2 = train_test_split(d2, y, test_size=0.3)

display(d2)

,LIMIT_BAL,SEX,EDUCATION,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,20000,2,2,24,2,2,-1,-1,-2,-2,...,689,0,0,0,0,689,0,0,0,0
1,120000,2,2,26,-1,2,0,0,0,2,...,2682,3272,3455,3261,0,1000,1000,1000,0,2000
2,90000,2,2,34,0,0,0,0,0,0,...,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
3,50000,2,2,37,0,0,0,0,0,0,...,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000
4,50000,1,2,57,-1,0,-1,0,0,0,...,35835,20940,19146,19131,2000,36681,10000,9000,689,679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,3,39,0,0,0,0,0,0,...,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000
29996,150000,1,3,43,-1,-1,-1,-1,0,0,...,3502,8979,5190,0,1837,3526,8998,129,0,0
29997,30000,1,2,37,4,3,2,-1,0,0,...,2758,20878,20582,19357,0,0,22000,4200,2000,3100
29998,80000,1,3,41,1,-1,0,0,0,-1,...,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804


Don't consider education or age

In [128]:
cols = ['EDUCATION','AGE']
d3 = d3.drop(cols,axis=1)
X_train3, X_test3, y_train3, y_test3 = train_test_split(d3, y, test_size=0.3)
display(d3)


,LIMIT_BAL,SEX,MARRIAGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,20000,2,1,2,2,-1,-1,-2,-2,3913,...,689,0,0,0,0,689,0,0,0,0
1,120000,2,2,-1,2,0,0,0,2,2682,...,2682,3272,3455,3261,0,1000,1000,1000,0,2000
2,90000,2,2,0,0,0,0,0,0,29239,...,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
3,50000,2,1,0,0,0,0,0,0,46990,...,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000
4,50000,1,1,-1,0,-1,0,0,0,8617,...,35835,20940,19146,19131,2000,36681,10000,9000,689,679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,1,0,0,0,0,0,0,188948,...,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000
29996,150000,1,2,-1,-1,-1,-1,0,0,1683,...,3502,8979,5190,0,1837,3526,8998,129,0,0
29997,30000,1,2,4,3,2,-1,0,0,3565,...,2758,20878,20582,19357,0,0,22000,4200,2000,3100
29998,80000,1,1,1,-1,0,0,0,-1,-1645,...,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804


Model that upsamples minority groups (this is dependent on the statistics we get)

In [129]:
# All Data
km = KMeans(n_clusters = 2) # binary classifications
km.fit(X_train)
preds = km.predict(X_test)
results = pd.DataFrame(X_test)
results['KMeans All Vars Predictions'] = preds
print(accuracy_score(preds, y_test))

km = KMeans(n_clusters = 2) # binary classifications
km.fit(X_train1)
preds = km.predict(X_test1)
results = pd.DataFrame(X_test1)
results['KMeans All Vars Predictions'] = preds
print(accuracy_score(preds, y_test1))

km = KMeans(n_clusters = 2) # binary classifications
km.fit(X_train2)
preds = km.predict(X_test2)
results = pd.DataFrame(X_test2)
results['KMeans All Vars Predictions'] = preds
print(accuracy_score(preds, y_test2))

km = KMeans(n_clusters = 2) # binary classifications
km.fit(X_train3)
preds = km.predict(X_test3)
results = pd.DataFrame(X_test3)
results['KMeans All Vars Predictions'] = preds
print(accuracy_score(preds, y_test3))


display(results)

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

0.6904


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,KMeans All Vars Predictions
2308,30000,1,2,2,25,0,0,0,0,0,...,12580,13716,14828,1500,2000,1500,1500,1500,2000,0
22404,150000,2,1,2,26,0,0,0,0,0,...,101581,77741,77264,4486,4235,3161,2647,2669,2669,0
23397,70000,2,3,1,32,0,0,0,0,0,...,69753,70111,70212,2431,3112,3000,2438,2500,2554,0
25058,130000,1,3,2,49,0,0,0,0,0,...,16898,11236,6944,1610,1808,7014,27,7011,4408,0
2664,50000,2,2,2,36,0,0,0,0,0,...,19574,20295,19439,2000,1500,1000,1800,0,1000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,40000,2,3,1,49,1,-2,-2,-2,-2,...,0,0,150,0,0,0,0,150,159,0
9355,330000,2,2,1,42,0,0,0,0,0,...,36710,40442,42283,10000,10000,5000,5000,5000,5000,0
28201,300000,2,3,2,48,1,3,2,-1,0,...,769,772,1722,781,1,769,3,1730,0,0
19705,100000,2,2,1,47,-1,-1,-1,-1,-1,...,6270,-396,-396,6666,0,6270,0,0,0,0


In [130]:
pca = PCA()
pca.fit(X_train)
reduced_train = pca.transform(X_test)
cov = pca.get_covariance()